# Dependency syntactic analysis

EstNLTK provides wrappers for two syntactic analysers: [MaltParser](http://www.maltparser.org/) and [VISLCG3 based syntactic analyser of Estonian](https://github.com/EstSyntax/EstCG).

MaltParser based syntactic analysis is distributed with EstNLTK and can be applied by default. VISLCG3 based syntactic analysis has a requirement that VISLCG3 must be installed into the system first.

Both analysers are using a common syntactic analysis tagset, which is introduced in https://korpused.keeleressursid.ee/syntaks/dokumendid/syntaksiliides_en.pdf.

## `VislTagger`: VISLCG3 syntactic analysis

VISL CG3 is a rule-based syntactic parser that has thousands of Estonian-specific handcrafted rules for tagging syntactic functions and dependencies. However, the parser needs more information than is given out by the standard morphological analyser (e.g. pronoun types, verb subcategorization, etc.), and its input needs to be in a different format. 

Therefore, to use VislTagger, we first need to add layer `morph_extended` to our Text object. This layer contains more detailed morphological analysis than the standard `morph_analysis` layer.


In [1]:
from estnltk import Text

In [2]:
text = Text('Ta on ise tee esimesel poolel.')
text.tag_layer(['morph_extended'])

Text(text='Ta on ise tee esimesel poolel.')

If we compare the standard `morph_analysis` layer with `morph_extended`, we can see that `morph_extended` has more refined labels under the 'form' attribute, as well as features like pronoun type, punctuation type, letter case, etc. Those extra features are needed because they are used in VislCG3 grammar rules. 

In addition, `morph_extended` layer is ambiguous, more so than the standard `morph_analysis` layer as it is more detailed. E.g. in our example sentence, we can see that the word 'on' gets 6 different analyses. As the first step of VislTagger is morphological disambiguation, the extra analyses will be removed and will not propagate to syntactic analysis layer.

In [3]:
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[AmbiguousSpan('Ta', [{'lemma': 'tema', 'root': 'tema', 'root_tokens': ('tema',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
AmbiguousSpan('on', [{'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
AmbiguousSpan('ise', [{'lemma': 'ise', 'root': 'ise', 'root_tokens': ('ise',), 'ending': '0', 'clitic': '', 'form': 'pl n', 'partofspeech': 'P'}, {'lemma': 'ise', 'root': 'ise', 'root_tokens': ('ise',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
AmbiguousSpan('tee', [{'lemma': 'tee', 'root': 'tee', 'root_tokens': ('tee',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
AmbiguousSpan('esimesel', [{'lemma': 'esimene', 'root': 'esimene', 'root_tokens': ('esimene',), 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'O'}]),
AmbiguousSpan('poolel', [{'lemma': 'pool', 'root': 'pool', 'root_tokens': ('pool',), 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'S'}]),
AmbiguousSpan('.', [{'lemma': '.', 'root': '.', 'root_tokens': ('.',), 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

In [4]:
text.morph_extended

Layer(name='morph_extended', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type', 'pronoun_type', 'letter_case', 'fin', 'verb_extension_suffix', 'subcat'), spans=SL[AmbiguousSpan('Ta', [{'lemma': 'tema', 'root': 'tema', 'root_tokens': ('tema',), 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ('ps3',), 'letter_case': 'cap', 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
AmbiguousSpan('on', [{'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}]),
AmbiguousSpan('ise', [{'lemma': 'ise', 'root': 'ise', 'root_tokens': ('ise',), 'ending': '0', 'clitic': '', 'form': 'pl nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ('pos', 'det', 'refl'), 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}, {'lemma': 'ise', 'root': 'ise', 'root_tokens': ('ise',), 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ('pos', 'det', 'refl'), 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
AmbiguousSpan('tee', [{'lemma': 'tee', 'root': 'tee', 'root_tokens': ('tee',), 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
AmbiguousSpan('esimesel', [{'lemma': 'esimene', 'root': 'esimene', 'root_tokens': ('esimene',), 'ending': 'l', 'clitic': '', 'form': 'ord sg ad roman', 'partofspeech': 'N', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}, {'lemma': 'esimene', 'root': 'esimene', 'root_tokens': ('esimene',), 'ending': 'l', 'clitic': '', 'form': 'ord sg ad l', 'partofspeech': 'N', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
AmbiguousSpan('poolel', [{'lemma': 'pool', 'root': 'pool', 'root_tokens': ('pool',), 'ending': 'l', 'clitic': '', 'form': 'com sg ad', 'partofspeech': 'S', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
AmbiguousSpan('.', [{'lemma': '.', 'root': '.', 'root_tokens': ('.',), 'ending'

In order to use VISLCG3 based syntactic analysis, the VISLCG3 parser must be installed into the system. The information about the parser is distributed in the [Constraint Grammar's Google Group](https://groups.google.com/forum/#!forum/constraint-grammar), and this is also the place to look for the most compact guide about getting & installing the [latest version of the parser](https://groups.google.com/forum/#!msg/constraint-grammar/hXsbzyyhIVI/nHXRnOomf9wJ).

By default, EstNLTK expects that the directory containing VISLCG3 parser's executable (vislcg3 in UNIX, vislcg3.exe in Windows) is accessible from system's environment variable PATH. If this requirement is satisfied, the EstNLTK should always be able to execute the parser and therefore, we can parse our example sentence as follows:

In [5]:
from estnltk.taggers import VislTagger

visl_tagger = VislTagger()
visl_tagger.tag(text)

text.visl

Layer(name='visl', attributes=('id', 'lemma', 'ending', 'partofspeech', 'subtype', 'mood', 'tense', 'voice', 'person', 'inf_form', 'number', 'case', 'polarity', 'number_format', 'capitalized', 'finiteness', 'subcat', 'clause_boundary', 'deprel', 'head'), spans=SL[AmbiguousSpan('Ta', [{'id': '1', 'lemma': 'tema', 'ending': '0', 'partofspeech': 'P', 'subtype': 'pers', 'mood': '_', 'tense': '_', 'voice': '_', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': 'cap', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@SUBJ', 'head': '2'}]),
AmbiguousSpan('on', [{'id': '2', 'lemma': 'ole', 'ending': '0', 'partofspeech': 'V', 'subtype': 'main', 'mood': 'indic', 'tense': 'pres', 'voice': 'ps', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': '_', 'polarity': 'af', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@FMV', 'head': '0'}]),
AmbiguousSpan('ise', [{'id': '3', 'lemma': 'ise', 'ending': '0', 'partofspeech': 'P', 'subtype': ['pos', 'det', 'refl'], 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@ADVL', 'head': '2'}]),
AmbiguousSpan('tee', [{'id': '4', 'lemma': 'tee', 'ending': '0', 'partofspeech': 'S', 'subtype': 'com', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@PRD', 'head': '2'}]),
AmbiguousSpan('esimesel', [{'id': '5', 'lemma': 'esimene', 'ending': 'l', 'partofspeech': 'N', 'subtype': 'ord', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'ad', 'polarity': '_', 'number_format': 'l', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@AN>', 'head': '6'}]),
AmbiguousSpan('poolel', [{'id': '6', 'lemma': 'pool', 'ending': 'l', 'partofspeech': 'S', 'subtype': 'com', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'ad', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': ['@<NN', '@ADVL'], 'head': '2'}]),
AmbiguousSpan('.', [{'id': '7', 'lemma': '.', 'ending': '_', 'partofspeech': 'Z', 'subtype': 'Fst', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': '_', 'case': '_', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': 'CLB', 'deprel': '_', 'head': '7'}])])

The parser assigns each word a syntactic label (`deprel`, (e.g. '@SUBJ' stands for subject, see the [documentation](https://korpused.keeleressursid.ee/syntaks/dokumendid/syntaksiliides_en.pdf) for details)) and its syntactic head (`head`) which is the id of its governing word in the sentence. NB! As can be seen from the example, the word id's start from 1 and not 0. The governing word index 0 marks that the current word is the root node of the tree.

As VislCG3 is based on [Constraint Grammar](http://visl.sdu.dk/constraint_grammar.html) formalism and first adds all the syntactic labels and then removes the ones that are not suitable based on constraints, it does leave some syntactic labels ambiguous as is seen with the word 'poolel' in our example sentence which gets both the analyses of a complement (@<NN) and adverbial (@ADVL). Despite this, each word still has only one syntactic head.

## `SyntaxDependencyRetagger` 

`SyntaxDependencyRetagger` adds `parent_span` and `children` attributes to the syntax layer.

Here the syntax layer is a layer that has at least `id` and `head` attributes.

The `parent_span` and `children` attributes help to navigate from a span to the parent and children of that span.

If `parent_span` or `children` attribute already exists in the layer then the values are updated. Therefore, to
    update the dependencies in the syntax layer first update the values of head attributes and then run this retagger.

In [6]:
from estnltk.taggers import SyntaxDependencyRetagger

SyntaxDependencyRetagger('visl').retag(text)

text.visl

Layer(name='visl', attributes=('id', 'lemma', 'ending', 'partofspeech', 'subtype', 'mood', 'tense', 'voice', 'person', 'inf_form', 'number', 'case', 'polarity', 'number_format', 'capitalized', 'finiteness', 'subcat', 'clause_boundary', 'deprel', 'head', 'parent_span', 'children'), spans=SL[AmbiguousSpan('Ta', [{'id': '1', 'lemma': 'tema', 'ending': '0', 'partofspeech': 'P', 'subtype': 'pers', 'mood': '_', 'tense': '_', 'voice': '_', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': 'cap', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@SUBJ', 'head': '2', 'parent_span': AmbiguousSpan('on', [{'id': '2', 'lemma': 'ole', 'ending': '0', 'partofspeech': 'V', 'subtype': 'main', 'mood': 'indic', 'tense': 'pres', 'voice': 'ps', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': '_', 'polarity': 'af', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@FMV', 'head': '0', 'parent_span': None, 'children': (..., AmbiguousSpan('ise', [{'id': '3', 'lemma': 'ise', 'ending': '0', 'partofspeech': 'P', 'subtype': ['pos', 'det', 'refl'], 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@ADVL', 'head': '2', 'parent_span': ..., 'children': ()}]), AmbiguousSpan('tee', [{'id': '4', 'lemma': 'tee', 'ending': '0', 'partofspeech': 'S', 'subtype': 'com', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@PRD', 'head': '2', 'parent_span': ..., 'children': ()}]), AmbiguousSpan('poolel', [{'id': '6', 'lemma': 'pool', 'ending': 'l', 'partofspeech': 'S', 'subtype': 'com', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'ad', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': ['@<NN', '@ADVL'], 'head': '2', 'parent_span': ..., 'children': (AmbiguousSpan('esimesel', [{'id': '5', 'lemma': 'esimene', 'ending': 'l', 'partofspeech': 'N', 'subtype': 'ord', 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'ad', 'polarity': '_', 'number_format': 'l', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@AN>', 'head': '6', 'parent_span': ..., 'children': ()}]),)}]))}]), 'children': ()}]),
AmbiguousSpan('on', [{'id': '2', 'lemma': 'ole', 'ending': '0', 'partofspeech': 'V', 'subtype': 'main', 'mood': 'indic', 'tense': 'pres', 'voice': 'ps', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': '_', 'polarity': 'af', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@FMV', 'head': '0', 'parent_span': None, 'children': (AmbiguousSpan('Ta', [{'id': '1', 'lemma': 'tema', 'ending': '0', 'partofspeech': 'P', 'subtype': 'pers', 'mood': '_', 'tense': '_', 'voice': '_', 'person': 'ps3', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': 'cap', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@SUBJ', 'head': '2', 'parent_span': ..., 'children': ()}]), AmbiguousSpan('ise', [{'id': '3', 'lemma': 'ise', 'ending': '0', 'partofspeech': 'P', 'subtype': ['pos', 'det', 'refl'], 'mood': '_', 'tense': '_', 'voice': '_', 'person': '_', 'inf_form': '_', 'number': 'sg', 'case': 'nom', 'polarity': '_', 'number_format': '_', 'capitalized': '_', 'finiteness': '_', 'subcat': '_', 'clause_boundary': '_', 'deprel': '@ADVL', 'head': '2', 'parent_span': ..., 'children': ()}]), AmbiguousSpan('tee', [{'id': '4', 'lemma': 'tee', 'ending': '0', 'partofs

The span `poolel` has a parent span and one child:

In [7]:
span = text.visl[5]
span

text,id,lemma,ending,partofspeech,subtype,mood,tense,voice,person,inf_form,number,case,polarity,number_format,capitalized,finiteness,subcat,clause_boundary,deprel,head,parent_span,children
poolel,6,pool,l,S,com,_,_,_,_,_,sg,ad,_,_,_,_,_,_,"['@<NN', '@ADVL']",2,"AmbiguousSpan('on', [{'id': '2', 'lemma': 'ole', 'ending': '0', 'partofspeech': ..., type: <class 'estnltk.layer.ambiguous_span.AmbiguousSpan'>, length: 1","(""AmbiguousSpan('esimesel', [{'id': '5', 'lemma': 'esimene', 'ending': 'l', 'par ..., type: <class 'tuple'>, length: 1"


To get the parent span write

In [8]:
span.annotations[0].parent_span

text,id,lemma,ending,partofspeech,subtype,mood,tense,voice,person,inf_form,number,case,polarity,number_format,capitalized,finiteness,subcat,clause_boundary,deprel,head,parent_span,children
on,2,ole,0,V,main,indic,pres,ps,ps3,_,sg,_,af,_,_,_,_,_,@FMV,0,None,"(""AmbiguousSpan('Ta', [{'id': '1', 'lemma': 'tema', 'ending': '0', 'partofspeech ..., type: <class 'tuple'>, length: 4"


To iterate over all children write

In [9]:
for child in span.annotations[0].children:
    child.display()

text,id,lemma,ending,partofspeech,subtype,mood,tense,voice,person,inf_form,number,case,polarity,number_format,capitalized,finiteness,subcat,clause_boundary,deprel,head,parent_span,children
esimesel,5,esimene,l,N,ord,_,_,_,_,_,sg,ad,_,l,_,_,_,_,@AN>,6,"AmbiguousSpan('poolel', [{'id': '6', 'lemma': 'pool', 'ending': 'l', 'partofspee ..., type: <class 'estnltk.layer.ambiguous_span.AmbiguousSpan'>, length: 1",()


# Maltparser

Maltparser is a data-driven parser and has been trained on [Estonian Dependency Treebank](https://github.com/EstSyntax/EDT). As its optimal data format for morphological analysis differs from the VislCG3 format, we first need to use  ConllMorphTagger to tag `conll_morph` layer onto our data. This is based on `morph_extended` layer.

In [10]:
from estnltk.taggers import ConllMorphTagger

tagger = ConllMorphTagger(output_layer='conll_morph',  # default: 'conll_morph'
                          morph_extended_layer='morph_extended'  # default: 'morph_extended'
                          )
tagger

name,output layer,output attributes,input layers
ConllMorphTagger,conll_morph,"('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc')","('morph_extended',)"


In [11]:
tagger.tag(text)

text.conll_morph

Layer(name='conll_morph', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[AmbiguousSpan('Ta', [{'id': '1', 'form': 'Ta', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'Ppers', 'feats': 'ps3|sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('on', [{'id': '2', 'form': 'on', 'lemma': 'ole', 'upostag': 'V', 'xpostag': 'V', 'feats': 'indic|pres|ps3|sg', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('ise', [{'id': '3', 'form': 'ise', 'lemma': 'ise', 'upostag': 'P', 'xpostag': 'P', 'feats': 'pos|det|refl|sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('tee', [{'id': '4', 'form': 'tee', 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': 'sg|nom', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('esimesel', [{'id': '5', 'form': 'esimesel', 'lemma': 'esimene', 'upostag': 'N', 'xpostag': 'A', 'feats': 'ord|sg|ad|l', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('poolel', [{'id': '6', 'form': 'poolel', 'lemma': 'pool', 'upostag': 'S', 'xpostag': 'S', 'feats': 'sg|ad', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}]),
AmbiguousSpan('.', [{'id': '7', 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': 'Fst', 'head': '_', 'deprel': '_', 'deps': '_', 'misc': '_'}])])

This way, we get a standard CONLL format that can also be converted into a string as follows.

In [12]:
from estnltk.taggers.syntax.conll_morph_to_str import *

print(conll_to_str(text))

1	Ta	tema	P	Ppers	ps3|sg|nom	_	_	_	_	
2	on	ole	V	V	indic|pres|ps3|sg	_	_	_	_	
3	ise	ise	P	P	pos|det|refl|sg|nom	_	_	_	_	
4	tee	tee	S	S	sg|nom	_	_	_	_	
5	esimesel	esimene	N	A	ord|sg|ad|l	_	_	_	_	
6	poolel	pool	S	S	sg|ad	_	_	_	_	
7	.	.	Z	Z	Fst	_	_	_	_	




This is useful for cases when we want to use Maltparser without the EstNLTK interface. However, we can also use MaltParser inside EstNLTK on our Text object and get the standard CONLL-format syntactic analysis:

In [13]:
from estnltk.taggers.syntax.maltparser import MaltParser

parser = MaltParser()
initial_output = parser.parse_text(text, return_type = 'text')

In [14]:
print( '\n'.join( initial_output) )

1	Ta	tema	P	Ppers	ps3|sg|nom	2	@SUBJ	_	_
2	on	ole	V	V	indic|pres|ps3|sg	0	ROOT	_	_
3	ise	ise	P	P	pos|det|refl|sg|nom	4	@NN>	_	_
4	tee	tee	S	S	sg|nom	2	@PRD	_	_
5	esimesel	esimene	N	A	ord|sg|ad|l	6	@AN>	_	_
6	poolel	pool	S	S	sg|ad	2	@ADVL	_	_
7	.	.	Z	Z	Fst	6	@Punc	_	_



In [15]:
with open("example.conll", "w") as fout:
    fout.write('\n'.join( initial_output))

If we want the analysis to be added to the Text object, we need to write it to a file first <font color='red'>(should be a better way of doing this)</font> and then we can import it as a Text object as described in the following subsection.

## Working with ConLL-X files

EstNLTK enables us to import data from ConLL-X files as Text objects with one or more syntactic analysis layers. To see how this works, let's use the example created in the previous subsection:

In [16]:
from estnltk.converters.conll_importer import conll_to_text

# Reading data from the file 'example.conll'
text = conll_to_text(file='example.conll', syntax_layer='malt_1')

In [17]:
text

Text(text='Ta on ise tee esimesel poolel .')

In [18]:
text.malt_1

Layer(name='malt_1', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('Ta', [{'id': 1, 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'Ppers', 'feats': OrderedDict([('ps3', ''), ('sg', ''), ('nom', '')]), 'head': 2, 'deprel': '@SUBJ', 'deps': None, 'misc': None, 'parent_span': Span('on', [{'id': 2, 'lemma': 'ole', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('indic', ''), ('pres', ''), ('ps3', ''), ('sg', '')]), 'head': 0, 'deprel': 'ROOT', 'deps': None, 'misc': None, 'parent_span': None, 'children': (..., Span('tee', [{'id': 4, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('nom', '')]), 'head': 2, 'deprel': '@PRD', 'deps': None, 'misc': None, 'parent_span': ..., 'children': (Span('ise', [{'id': 3, 'lemma': 'ise', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pos', ''), ('det', ''), ('refl', ''), ('sg', ''), ('nom', '')]), 'head': 4, 'deprel': '@NN>', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]),)}]), Span('poolel', [{'id': 6, 'lemma': 'pool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('ad', '')]), 'head': 2, 'deprel': '@ADVL', 'deps': None, 'misc': None, 'parent_span': ..., 'children': (Span('esimesel', [{'id': 5, 'lemma': 'esimene', 'upostag': 'N', 'xpostag': 'A', 'feats': OrderedDict([('ord', ''), ('sg', ''), ('ad', ''), ('l', '')]), 'head': 6, 'deprel': '@AN>', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]), Span('.', [{'id': 7, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict([('Fst', '')]), 'head': 6, 'deprel': '@Punc', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]))}]))}]), 'children': ()}]),
Span('on', [{'id': 2, 'lemma': 'ole', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('indic', ''), ('pres', ''), ('ps3', ''), ('sg', '')]), 'head': 0, 'deprel': 'ROOT', 'deps': None, 'misc': None, 'parent_span': None, 'children': (Span('Ta', [{'id': 1, 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'Ppers', 'feats': OrderedDict([('ps3', ''), ('sg', ''), ('nom', '')]), 'head': 2, 'deprel': '@SUBJ', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]), Span('tee', [{'id': 4, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('nom', '')]), 'head': 2, 'deprel': '@PRD', 'deps': None, 'misc': None, 'parent_span': ..., 'children': (Span('ise', [{'id': 3, 'lemma': 'ise', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pos', ''), ('det', ''), ('refl', ''), ('sg', ''), ('nom', '')]), 'head': 4, 'deprel': '@NN>', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]),)}]), Span('poolel', [{'id': 6, 'lemma': 'pool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('ad', '')]), 'head': 2, 'deprel': '@ADVL', 'deps': None, 'misc': None, 'parent_span': ..., 'children': (Span('esimesel', [{'id': 5, 'lemma': 'esimene', 'upostag': 'N', 'xpostag': 'A', 'feats': OrderedDict([('ord', ''), ('sg', ''), ('ad', ''), ('l', '')]), 'head': 6, 'deprel': '@AN>', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]), Span('.', [{'id': 7, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict([('Fst', '')]), 'head': 6, 'deprel': '@Punc', 'deps': None, 'misc': None, 'parent_span': ..., 'children': ()}]))}]))}]),
Span('ise', [{'id': 3, 'lemma': 'ise', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pos', ''), ('det', ''), ('refl', ''), ('sg', ''), ('nom', '')]), 'head': 4, 'deprel': '@NN>', 'deps': None, 'misc': None, 'parent_span': Span('tee', [{'id': 4, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('nom', '')]), 'head': 2, 'deprel': '@PRD', 'deps': None, 'misc': None, 'parent_span': Span('on', [{'id': 2, 'lemma': 'ole', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('indic', ''), ('pres', ''), ('ps3', ''), ('sg', '')]), 'head': 0, 'deprel': 'ROOT', 'deps': None, 'misc

We can also add more syntax layers from CONLL files. This is useful if we have parsed our data with different parsing models and/or have gold standard and want to compare the annotations.

In [19]:
# Let's create another file, although it will have the same content as the first one
with open("example2.conll", "w") as fout:
    fout.write('\n'.join( initial_output))

In [20]:
from estnltk.converters.conll_importer import add_layer_from_conll

# Adding the analysis in the second file to the Text object that we already created
add_layer_from_conll(file='example2.conll', text=text, syntax_layer='malt_2')

Text(text='Ta on ise tee esimesel poolel .')

We can also iterate over sentences and words and compare their attributes. 

In [21]:
malt_1 = text.malt_1
malt_2 = text.malt_2

for sentence in text.sentences:
    for word in sentence:
        if malt_1.get(word).deprel == malt_2.get(word).deprel:
            print(malt_1.get(word).text, malt_2.get(word).text)

Ta Ta
on on
ise ise
tee tee
esimesel esimesel
poolel poolel
. .




## Labeled Attachment Score

Now that we have two syntactic analysis layers on our Text object, we probably want to compare them computationally as well. Labeled Attachment Score (LAS) is a standard evaluation metric in dependency syntax. It is the ratio of words that are assigned both the correct syntactic head and the correct dependency label and varies between 0 and 1. 

In [22]:
from estnltk.syntax.scoring import las_score

To understand better, let's change the second layer a little bit:

In [23]:
text.malt_2[3].deprel = '@X'

text.malt_1[3].deprel != text.malt_2[3].deprel

True

Now we can calculate LAS between the two syntax layers:

In [24]:
las_score(layer_a=text.malt_1,
          layer_b=text.malt_2
          )

0.8571428571428571

We can also just calculate the score for some parts of our Text object, e.g. to compare the first 4 spans or to compare the spans starting from the 5th:

In [25]:
las_score(text.malt_1, text.malt_2, 0, 4)

0.75

In [26]:
las_score(text.malt_1, text.malt_2, 4)

1.0

In addition, we can tag sliding LAS scores to be able to compare different text segments.

In [27]:
from estnltk.taggers import SyntaxLasTagger

tagger = SyntaxLasTagger('malt_1', 'malt_2', window=3)
tagger

SyntaxLasTagger(input_layers=('malt_1', 'malt_2'), output_layer=las, output_attributes=('deprel_sequence', 'score'), window=3)

In [28]:
tagger.tag(text)
text.las

Layer(name='las', attributes=('deprel_sequence', 'score'), spans=SL[EnvelopingSpan(['Ta'], [{'deprel_sequence': ('@SUBJ',), 'score': 1.0}]),
EnvelopingSpan(['Ta', 'on'], [{'deprel_sequence': ('@SUBJ', 'ROOT'), 'score': 1.0}]),
EnvelopingSpan(['Ta', 'on', 'ise'], [{'deprel_sequence': ('@SUBJ', 'ROOT', '@NN>'), 'score': 1.0}]),
EnvelopingSpan(['on', 'ise', 'tee'], [{'deprel_sequence': ('ROOT', '@NN>', '@PRD'), 'score': 0.6666666666666666}]),
EnvelopingSpan(['ise', 'tee', 'esimesel'], [{'deprel_sequence': ('@NN>', '@PRD', '@AN>'), 'score': 0.6666666666666666}]),
EnvelopingSpan(['tee', 'esimesel', 'poolel'], [{'deprel_sequence': ('@PRD', '@AN>', '@ADVL'), 'score': 0.6666666666666666}]),
EnvelopingSpan(['esimesel', 'poolel', '.'], [{'deprel_sequence': ('@AN>', '@ADVL', '@Punc'), 'score': 1.0}]),
EnvelopingSpan(['poolel', '.'], [{'deprel_sequence': ('@ADVL', '@Punc'), 'score': 1.0}]),
EnvelopingSpan(['.'], [{'deprel_sequence': ('@Punc',), 'score': 1.0}])])

`SyntaxLasTagger` adds metadata to the output layer. The `aggregate_deprel_sequences` field of the meta lists all LAS scores for every `deprel` sequence encountered.

In [29]:
text.las.meta

{'aggregate_deprel_sequences': {('@SUBJ',): [1.0],
  ('@SUBJ', 'ROOT'): [1.0],
  ('@SUBJ', 'ROOT', '@NN>'): [1.0],
  ('ROOT', '@NN>', '@PRD'): [0.6666666666666666],
  ('@NN>', '@PRD', '@AN>'): [0.6666666666666666],
  ('@PRD', '@AN>', '@ADVL'): [0.6666666666666666],
  ('@AN>', '@ADVL', '@Punc'): [1.0],
  ('@ADVL', '@Punc'): [1.0],
  ('@Punc',): [1.0]}}